In [1]:
import pandas as pd

# Load the CSV file
data_path = './data_cleaned_chatbot_data.csv'
df = pd.read_csv(data_path)

# Check for missing values before processing
print("Missing values before filling:\n", df.isnull().sum())

# Define the placeholder text for missing responses
placeholder_response = "I'm here to help. Can you tell me more?"

# Fill missing values in the 'Response' column with the placeholder text
df['Response'] = df['Response'].fillna(placeholder_response)

# Check for missing values after processing
print("Missing values after filling:\n", df.isnull().sum())

# Save the cleaned data to a new file
cleaned_data_path = "cleaned_chatbot_data.csv"
df.to_csv(cleaned_data_path, index=False)

print(f"Cleaned data saved to: {cleaned_data_path}")

Missing values before filling:
 Context     0
Response    4
dtype: int64
Missing values after filling:
 Context     0
Response    0
dtype: int64
Cleaned data saved to: cleaned_chatbot_data.csv


In [2]:
!pip install datasets

import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer

# Load and preprocess the dataset
df = pd.read_csv("cleaned_chatbot_data.csv")
df["task"] = "detect depression: " + df["Context"]
df = df.rename(columns={"task": "input_text", "Response": "target_text"})

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1)

# Load T5 tokenizer and model
model_name = "t5-small"  # Choose from t5-small, t5-base, etc.
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Tokenize the dataset
def preprocess(data):
    inputs = tokenizer(data["input_text"], max_length=512, truncation=True, padding="max_length")
    targets = tokenizer(data["target_text"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized_dataset = dataset.map(preprocess, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./t5-fine-tuned",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    save_total_limit=1,
    fp16=True,  # Enable if using GPU with 16-bit precision
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
)

# Fine-tune the model
trainer.train()

# Save the model
trainer.save_model("./t5-fine-tuned")
tokenizer.save_pretrained("./t5-fine-tuned")

print("Fine-tuning complete. Model saved!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

...

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,3.093992
2,3.300100,3.032531
3,3.300100,3.013973


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


Fine-tuning complete. Model saved!
